# Prática: Redes Neurais Convolucionais

Vamos agora implementar a rede [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), uma das redes que trouxeram todo esse interesse para a área de *deep learning*.





In [3]:
import time, os, sys, numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torchsummary import summary


import time, os, sys, numpy as np

# Test if GPU is avaliable, if not, use cpu instead
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids= list(range(n))

In [4]:
def load_data_cifar10(batch_size, resize=None, root=os.path.join(
        '~', '.pytorch', 'datasets', 'cifar10')):
    """Download the Cifar10 dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [torchvision.transforms.Resize(resize)]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    mnist_train = torchvision.datasets.CIFAR10(root=root, train=True,download=True,transform=transformer)
    mnist_test = torchvision.datasets.CIFAR10(root=root, train=False,download=True,transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = torch.utils.data.DataLoader(mnist_train,
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test,
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter


# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss):
    """Evaluate accuracy of a model on the given data set."""

    acc_sum, n, l = torch.Tensor([0]), 0, 0
    net.eval()
    with torch.no_grad():
      for X, y in data_iter:
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y).sum()
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
          n += y.size()[0]

    return acc_sum.item() / n, l.item() / len(data_iter)

# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss,
                   num_epochs):
    print('training on', device)
    for epoch in range(num_epochs):
        net.train()
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            trainer.zero_grad()
            l = loss(y_hat, y).sum()
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size()[0]
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss,
                 test_acc, time.time() - start))

## AlexNet

Agora já temos todo o conhecimento necessário para implementar nossa primeira arquitetura moderna.
Vamos implementar a [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), uma das arquiteturas mais famosas dessa nova onda de rede neurais.

<p align="center">
  <img width=700 src="https://www.researchgate.net/profile/Jaime_Gallego2/publication/318168077/figure/fig1/AS:578190894927872@1514862859810/AlexNet-CNN-architecture-layers.png">
</p>

<p align="center">
  <img width=700 src="https://miro.medium.com/max/700/1*vXBvV_Unz3JAxytc5iSeoQ.png">
</p>

Lembre-se que, após cada camada de convolução e linear, há uma ativação não linear ReLU.

In [5]:
root = os.path.join('~', '.pytorch', 'datasets', 'cifar10')

In [17]:
# Implementa sua rede neural aqui
# Voce pode implementar como uma classe (pois fica mais organizado e é mais facilitado para um debug da rede)
# Dica: utilize blocos de sequential para diminuir a complexidade da função de forward
# rede baseada na AlexNet
import torch.nn as nn

class AlexNet(nn.Module):
    def __init__(self, input_channels, classes=10, **kwargs):
        super(AlexNet, self).__init__(**kwargs)

        self.cnnLayers = nn.Sequential(
            nn.Conv2d(in_channels = input_channels ,out_channels=96,kernel_size=11, stride=4, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
            nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2,padding=0)
        )

        self.linearLayers = nn.Sequential(
            nn.Linear(in_features=9216, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features= classes),
        )



    def forward(self, x):
        # defina aqui a função de forward
        x = self.cnnLayers(x)
        x = x.view(x.size(0),-1)
        x = self.linearLayers(x)
        return x

# parâmetros: número de epochs, learning rate (ou taxa de aprendizado),
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001

# Instanciar a rede (lembre-se de passar os parâmetros input_channels e classes)
net = AlexNet(input_channels=3, classes=10)
# Sending model to device
net.to(device)

print(summary(net,(3,227,227))) # visualizar o número de parâmetros da rede e output de cada camada

# função de custo (ou loss)
# defina a função de custo
loss = nn.CrossEntropyLoss()

# carregamento do dado: cifar10
train_iter, test_iter = load_data_cifar10(batch_size, resize=227, root=root)

# definindo o otimizador
trainer = optim.Adam(net.parameters(), lr=lr, weight_decay=wd_lambda)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
            Conv2d-4          [-1, 256, 27, 27]         614,656
              ReLU-5          [-1, 256, 27, 27]               0
         MaxPool2d-6          [-1, 256, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         885,120
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]       1,327,488
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         884,992
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
           Linear-14                 [-

100%|██████████| 170M/170M [00:05<00:00, 28.9MB/s]


training on cuda


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


epoch 1, train loss 1.9336, train acc 0.274, test loss 1.5466, test acc 0.431, time 95.5 sec
epoch 2, train loss 1.4853, train acc 0.454, test loss 1.3721, test acc 0.498, time 97.0 sec
epoch 3, train loss 1.3342, train acc 0.517, test loss 1.2913, test acc 0.540, time 96.3 sec
epoch 4, train loss 1.2053, train acc 0.568, test loss 1.1754, test acc 0.587, time 97.7 sec
epoch 5, train loss 1.1043, train acc 0.606, test loss 1.1623, test acc 0.587, time 97.5 sec
epoch 6, train loss 0.9995, train acc 0.645, test loss 1.0607, test acc 0.627, time 97.4 sec
epoch 7, train loss 0.9111, train acc 0.678, test loss 1.0313, test acc 0.645, time 97.4 sec
epoch 8, train loss 0.8229, train acc 0.706, test loss 1.0221, test acc 0.649, time 96.7 sec
epoch 9, train loss 0.7544, train acc 0.734, test loss 1.1386, test acc 0.633, time 98.0 sec
epoch 10, train loss 0.6734, train acc 0.761, test loss 1.0385, test acc 0.656, time 96.3 sec
epoch 11, train loss 0.5993, train acc 0.788, test loss 1.0767, test 